## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


In [75]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import os
%matplotlib qt

In [76]:
def calibrate_camer():
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    cwd = os.getcwd()
    # Make a list of calibration images
    images = glob.glob(cwd + './camera_cal/calibration*.jpg')
    
    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    img = cv2.imread(cwd + './camera_cal/calibration3.jpg')
    return cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)

# TODO: Write a function that takes an image, object points, and image points
# performs the camera calibration, image distortion correction and 
# returns the undistorted image
def undistort_img(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def cal_perspective_transform_mtx(img, nx, ny, mtx, dist):
    # Use the OpenCV undistort() function to remove distortion
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
#     plt.imshow(undist)
    
    # Convert undistorted image to grayscale
    gray = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)
    
#     plt.imshow(gray)
    
    # Search for corners in the grayscaled image
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    M = []
    if ret == True:
        # If we found corners, draw them! (just for fun)
#         cv2.drawChessboardCorners(undist, (nx, ny), corners, ret)
        # Choose offset from image corners to plot detected corners
        # This should be chosen to present the result at the proper aspect ratio
        # My choice of 100 pixels is not exact, but close enough for our purpose here
        offset = 100 # offset for dst points
        # Grab the image shape
        img_size = (gray.shape[1], gray.shape[0])

        # For source points I'm grabbing the outer four detected corners
        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        # For destination points, I'm arbitrarily choosing some points to be
        # a nice fit for displaying our warped result 
        # again, not exact, but close enough for our purposes
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                                     [img_size[0]-offset, img_size[1]-offset], 
                                     [offset, img_size[1]-offset]])
        # Given src and dst points, calculate the perspective transform matrix
        M = cv2.getPerspectiveTransform(src, dst)

    # Return the resulting image and matrix
    return M

# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output

# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result:
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def combining_thresholds(img, orient='x', thresh_min=0, thresh_max=255,sobel_kernel=3, mag_thresh=(0, 255), thresh=(0, np.pi/2)
):
    # Choose a Sobel kernel size
    ksize = 3 # Choose a larger odd number to smooth gradient measurements

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(image, 'x', ksize, thresh_min, thresh_max)
    grady = abs_sobel_thresh(image, 'y', ksize, thresh_min, thresh_max)
    mag_binary = mag_thresh(image, ksize, mag_thresh)
    dir_binary = dir_threshold(image, ksize, thresh)
    
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    # Return the binary image
    return combined

# Define a function that thresholds the S-channel of HLS
def hls_select(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

# Edit this function to create your own pipeline.
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
#     color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
#     return color_binary

    return s_binary

def warper(img, src, dst):

    # Compute and apply perpective transform
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)  # keep same size as input image

    return warped

def hist(img):
    # Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = img[img.shape[0]//2:,:]

    # Sum across image pixels vertically - make sure to set an `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half, axis=0)
    
    return histogram

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def cal_fit(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit

def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')

    return out_img

def fit_poly(img_shape, leftx, lefty, rightx, righty):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

def search_around_poly(binary_warped):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### hzf
    # Find our lane pixels first
    leftx1, lefty1, rightx1, righty1, out_img1 = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty1, leftx1, 2)
    right_fit = np.polyfit(righty1, rightx1, 2)
    ### hzf
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result

def generate_data():
    '''
    Generates fake data to use for calculating lane curvature.
    In your own project, you'll ignore this function and instead
    feed in the output of your lane detection algorithm to
    the lane curvature calculation.
    '''
    # Set random seed number so results are consistent for grader
    # Comment this out if you'd like to see results on different random data!
    np.random.seed(0)
    # Generate some fake data to represent lane-line pixels
    ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
    quadratic_coeff = 3e-4 # arbitrary quadratic coefficient
    # For each y position generate random x position within +/-50 pix
    # of the line base position in each case (x=200 for left, and x=900 for right)
    leftx = np.array([200 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
                                    for y in ploty])
    rightx = np.array([900 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
                                    for y in ploty])

    leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
    rightx = rightx[::-1]  # Reverse to match top-to-bottom in y


    # Fit a second order polynomial to pixel positions in each fake lane line
    left_fit = np.polyfit(ploty, leftx, 2)
    right_fit = np.polyfit(ploty, rightx, 2)
    
    return ploty, left_fit, right_fit


def my_measure_curvature_pixels(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])

    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

def my_measure_vichle_position(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    Ymax = binary_warped.shape[0]
    Xleft = left_fit[0] * Ymax * Ymax + left_fit[1] * Ymax + left_fit[2]
    Xright = right_fit[0] * Ymax * Ymax + right_fit[1] * Ymax + right_fit[2]

    
    Xcenter = (Xleft + Xright) / 2
    image_x_center = binary_warped.shape[1] / 2
    
    print(Ymax)
    print(image_x_center)
    
    
    print(Xleft)
    print(Xright)
    print(Xcenter)
    
    
    offset = (image_x_center - Xcenter) * xm_per_pix
    
    print(offset)
    
    return offset    
        
def my_measure_curvature_real(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
   
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
  
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
#     # Fit a second order polynomial to each using `np.polyfit`
#     left_fit = np.polyfit(lefty, leftx, 2)
#     right_fit = np.polyfit(righty, rightx, 2)

    # Fit a second order polynomial to pixel positions in each fake lane line
    # Fit new polynomials to x,y in world space
#     left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
#     right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
       
    return left_curverad, right_curverad  

def my_measure_curvature_pixels2(ploty, left_fit, right_fit):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''  
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad
    
def measure_curvature_pixels():
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    ploty, left_fit, right_fit = generate_data()
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

def generate_data_real(ym_per_pix, xm_per_pix):
    '''
    Generates fake data to use for calculating lane curvature.
    In your own project, you'll ignore this function and instead
    feed in the output of your lane detection algorithm to
    the lane curvature calculation.
    '''
    # Set random seed number so results are consistent for grader
    # Comment this out if you'd like to see results on different random data!
    np.random.seed(0)
    # Generate some fake data to represent lane-line pixels
    ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
    quadratic_coeff = 3e-4 # arbitrary quadratic coefficient
    # For each y position generate random x position within +/-50 pix
    # of the line base position in each case (x=200 for left, and x=900 for right)
    leftx = np.array([200 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
                                    for y in ploty])
    rightx = np.array([900 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
                                    for y in ploty])

    leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
    rightx = rightx[::-1]  # Reverse to match top-to-bottom in y

    # Fit a second order polynomial to pixel positions in each fake lane line
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    
    return ploty, left_fit_cr, right_fit_cr
    
def measure_curvature_real():
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    ploty, left_fit_cr, right_fit_cr = generate_data(ym_per_pix, xm_per_pix)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad


def my_draw(image, undist, warped, Minv,left_fitx,right_fitx,ploty):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result


In [77]:
ret, mtx, dist, rvecs, tvecs = calibrate_camer()

print(ret)
# print(mtx)
# print(dist)
# print(rvecs)
# print(tvecs)

s1 = [180,719]
s2 = [555, 470]
s3 = [725, 470]
s4 = [1130,719]

d1 = [280,719]
d2 = [280,  0]
d3 = [1000,  0]
d4 = [1000, 719]

src = np.array([s1,s2,s3,s4],dtype = "float32")
dst = np.array([d1,d2,d3,d4],dtype = "float32")

1.0298153371058987


In [78]:
def my_cal_perspective_transform_mtx():
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    return M, Minv

In [79]:
cwd = os.getcwd()
img = cv2.imread(cwd + '/camera_cal/calibration3.jpg')

cv2.imshow('calibration3.jpg',img)

img_undistort = undistort_img(img, mtx, dist)

cv2.imshow('img_undistort', img_undistort)

perspective_transform_mtx = cal_perspective_transform_mtx(img, 9, 6, mtx, dist)

print(perspective_transform_mtx)

print(img.shape)

warped = cv2.warpPerspective(img_undistort, perspective_transform_mtx, img.shape[1::-1], cv2.INTER_LINEAR)

# cv2.imshow('warped', warped)


[[ 1.23176607e+00  2.89559832e-01 -1.55916586e+02]
 [-2.60900799e-02  1.31677308e+00  2.97882835e+01]
 [-6.38198199e-05  4.89794342e-04  1.00000000e+00]]
(720, 1280, 3)


In [80]:
def my_pipline(img):
    img_undistort = undistort_img(img, mtx, dist)
    
    img_bin_sobel =  abs_sobel_thresh(img_undistort, orient='x', thresh_min=10, thresh_max=200)   

    img_bin_mag =  mag_thresh(img_undistort, sobel_kernel=3, mag_thresh=(20, 200))

    img_bin_dir =  dir_threshold(img_undistort, sobel_kernel=3, thresh=(0.7, 1.3))

    img_bin_hls =  hls_select(img_undistort, thresh=(25, 200))

    combined = np.zeros_like(img_bin_sobel)
    combined[((img_bin_sobel == 1) & (img_bin_mag == 1)) & ((img_bin_dir == 1) & (img_bin_hls == 1))] = 1

    img_cpy = np.copy(img_undistort)

    M, Minv = my_cal_perspective_transform_mtx()

    colored_comb_bin = np.dstack((combined, combined, combined)) * 255

    img_warper = warper(combined, src, dst)

    colored_warped = np.dstack((img_warper, img_warper, img_warper)) * 255

    histogram = hist(colored_warped)

    leftx, lefty, rightx, righty, out_img = find_lane_pixels(img_warper* 255) 

    img_fit = fit_polynomial(img_warper* 255)

    left_fit, right_fit, ploty = fit_poly(img_warper.shape, leftx, lefty, rightx, righty)

    result = search_around_poly(img_warper)

    left_curverad, right_curverad = my_measure_curvature_real(img_warper)
    
    offset = my_measure_vichle_position(img_warper)

    print(left_curverad)
    print(right_curverad)
    print(offset)

    img_final = my_draw(img, img_undistort, img_warper, Minv,left_fit,right_fit,ploty)

    font = cv2.FONT_HERSHEY_SIMPLEX

    imgzi = cv2.putText(img_final, 'Radius of Curvature = %f(m)' %(left_curverad) , (50, 50), font, 1.2, (255, 255, 255), 2)
    
    if offset > 0 :
        imgzi = cv2.putText(img_final, 'Vehicle is %fm right of center' %(offset) , (50, 50), font, 1.2, (255, 255, 255), 2)
    else:
        imgzi = cv2.putText(img_final, 'Vehicle is %fm left of center' %(abs(offset)) , (50, 100), font, 1.2, (255, 255, 255), 2)
        
    return img_final


In [81]:
cwd = os.getcwd()

img = mpimg.imread(cwd + '/test_images/straight_lines2.jpg')
# img = mpimg.imread(cwd + '/test_images/test2.jpg')

# plt.imshow(img)

img_undistort = undistort_img(img, mtx, dist)

# plt.imshow(img)

plt.imshow(img_undistort)

plt.show()


In [83]:
img_bin_sobel =  abs_sobel_thresh(img_undistort, orient='x', thresh_min=10, thresh_max=200)

print(img_bin_sobel)

# plt.imshow(img_bin_sobel, cmap='gray')

img_bin_mag =  mag_thresh(img_undistort, sobel_kernel=3, mag_thresh=(20, 200))

# plt.imshow(img_bin_mag, cmap='gray')

img_bin_dir =  dir_threshold(img_undistort, sobel_kernel=3, thresh=(0.7, 1.3))

# plt.imshow(img_bin_dir, cmap='gray')

img_bin_hls =  hls_select(img_undistort, thresh=(25, 200))

# plt.imshow(img_bin_hls, cmap='gray')

combined = np.zeros_like(img_bin_sobel)
combined[((img_bin_sobel == 1) & (img_bin_mag == 1)) & ((img_bin_dir == 1) & (img_bin_hls == 1))] = 1
# combined[((img_bin_sobel == 1) )] = 1

# plt.imshow(combined, cmap='gray')


# (120,539),(480, 310), (485, 310), (900,539)
# img_cpy = np.copy(img_undistort)
img_cpy = np.copy(img_undistort)

# plt.imshow(img_cpy)

print(img_cpy.shape)

s1 = [180,719]
s2 = [555, 470]
s3 = [725, 470]
s4 = [1130,719]

d1 = [280,719]
d2 = [280,  0]
d3 = [1000,  0]
d4 = [1000, 719]

w = 1
# cv2.line(img_cpy, (180,719), (550, 470), (255,0,0),w) 
# cv2.line(img_cpy, (550, 470), (740, 470), (255,0,0),w) 
# cv2.line(img_cpy, (740, 470), (1130,719), (255,0,0),w) 

cv2.line(img_cpy, s1, s2, (255,0,0),w) 
cv2.line(img_cpy, s2, s3, (255,0,0),w) 
cv2.line(img_cpy, s3, s4, (255,0,0),w) 

plt.imshow(img_cpy)

# src = np.array([[180,719],[550,470],[740, 470],[1130,719]],dtype = "float32")
# dst = np.array([[280,719],[280,  0],[1000,  0],[1000, 719]],dtype = "float32")

src = np.array([s1,s2,s3,s4],dtype = "float32")
dst = np.array([d1,d2,d3,d4],dtype = "float32")

M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
    
img_warper = warper(img_cpy, src, dst)

# plt.imshow(img_warper)

# img_warper = cv2.warpPerspective(img_cpy, perspective_transform_mtx, img_cpy.shape[1::-1], cv2.INTER_LINEAR)

# plt.imshow(img_warper)

colored_comb_bin = np.dstack((combined, combined, combined)) * 255

# cv2.line(colored_comb_bin, s1, s2, (255,0,0),w) 
# cv2.line(colored_comb_bin, s2, s3, (255,0,0),w) 
# cv2.line(colored_comb_bin, s3, s4, (255,0,0),w) 

# plt.imshow(colored_comb_bin, cmap='gray')

img_warper = warper(combined, src, dst)

# plt.imshow(img_warper, cmap='gray')

colored_warped = np.dstack((img_warper, img_warper, img_warper)) * 255

# plt.imshow(colored_warped, cmap='gray')

histogram = hist(colored_warped)

# plt.plot(histogram)

# leftx, lefty, rightx, righty, out_img = find_lane_pixels(img_warper)
leftx, lefty, rightx, righty, out_img = find_lane_pixels(img_warper* 255) 

print(leftx)
print(lefty)
print(rightx)
print(righty)

plt.imshow(out_img)

img_fit = fit_polynomial(img_warper* 255)

# plt.imshow(img_fit)

left_fit, right_fit, ploty = fit_poly(img_warper.shape, leftx, lefty, rightx, righty)

result = search_around_poly(img_warper)

# plt.imshow(result)

# left_curverad, right_curverad = measure_curvature_pixels()

left_curverad, right_curverad = my_measure_curvature_pixels(img_warper)

print(left_curverad)
print(right_curverad)

left_curverad, right_curverad = my_measure_curvature_real(img_warper)

print(left_curverad)
print(right_curverad)

# plt.imshow(img)
# plt.imshow(img_undistort)
# plt.imshow(img_warper)

# img_final = my_draw(img, img_undistort, img_warper, Minv,left_fitx,right_fitx,ploty)
img_final = my_draw(img, img_undistort, img_warper, Minv,left_fit,right_fit,ploty)

# img = cv2.imread('caijian.jpg')
font = cv2.FONT_HERSHEY_SIMPLEX

imgzi = cv2.putText(img_final, 'Curve Radius: left: %fm, right =%fm' %(left_curverad,right_curverad) , (50, 50), font, 1.2, (255, 255, 255), 2)

# plt.imshow(img_final)

plt.show()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(720, 1280, 3)
[304 320 321 ... 335 336 337]
[640 640 640 ... 159 159 159]
[971 972 988 ... 989 990 991]
[640 640 640 ...  79  79  79]
16453.63691348033
35613.25028830576
5404.234034427476
11695.209137806009


In [84]:
img_result = my_pipline(img)
print(img_result.shape)
plt.imshow(img_result)

plt.show()


720
640.0
313.66663293035873
977.9029651836306
645.7847990569946
-0.030576795015543116
5404.234034427476
11695.209137806009
-0.030576795015543116
(720, 1280, 3)


In [85]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [86]:
white_output = cwd + '/output_video/P2.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip(cwd + "/project_video.mp4")
white_clip = clip1.fl_image(my_pipline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

720
640.0
359.23706288171536
1039.7822514260647
699.50965715389
-0.3145510449562756
1367.7282463768483
1005.3271094970519
-0.3145510449562756
Moviepy - Building video D:\hzf\udacity\project\CarND-Advanced-Lane-Lines/output_video/P2.mp4.
Moviepy - Writing video D:\hzf\udacity\project\CarND-Advanced-Lane-Lines/output_video/P2.mp4



t:   0%|                                                                    | 2/1260 [00:00<04:05,  5.13it/s, now=None]

720
640.0
359.23706288171536
1039.7822514260647
699.50965715389
-0.3145510449562756
1367.7282463768483
1005.3271094970519
-0.3145510449562756


t:   0%|▏                                                                   | 3/1260 [00:00<06:01,  3.48it/s, now=None]

720
640.0
360.0447607272859
1036.0185830373068
698.0316718822963
-0.3067388370921378
1264.5163627442682
890.3434964176457
-0.3067388370921378


...

720
640.0
376.6874121015338
1071.5890739008964
724.1382430012152
-0.44473071300642314
24615.00532367592
3693.2497520316274
-0.44473071300642314


t: 100%|█████████████████████████████████████████████████████████████████| 1260/1260 [09:38<00:00,  2.17it/s, now=None]

720
640.0
375.81489294012266
1062.4970542365727
719.1559735883477
-0.41839586039555215
12838.12286586808
3448.342198952954
-0.41839586039555215


720
640.0
370.9787355800847
1070.7499778445192
720.864356712302
-0.42742588547931043
16777.30194420611
4634.381796380513
-0.42742588547931043


Moviepy - Done !
Moviepy - video ready D:\hzf\udacity\project\CarND-Advanced-Lane-Lines/output_video/P2.mp4
Wall time: 9min 40s


In [88]:
print(white_output)
# HTML("""
# <video width="1280" height="720" controls>
#   <source src="{0}">
# </video>
# """.format(white_output))

HTML("""
<video width="1280" height="720" controls>
  <source src="D:\\hzf\\udacity\\project\\CarND-Advanced-Lane-Lines\\output_video\\P2.mp4">
</video>
""")

D:\hzf\udacity\project\CarND-Advanced-Lane-Lines/output_video/P2.mp4


In [20]:
img = pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100))

print(img)
plt.imshow(img, cmap='gray')
# plt.imshow(img)

plt.show()

[[0 1 0 ... 1 1 1]
 [0 1 0 ... 1 1 1]
 [0 1 0 ... 1 1 0]
 ...
 [0 1 0 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]]


In [68]:

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
# print(objp)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
# print(objp)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

cwd = os.getcwd()
print(cwd)
img = cv2.imread(cwd + '/camera_cal/calibration3.jpg')
# print(img)
# cv2.imshow('calibration3',img)

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

print(gray.shape)

# cv2.imshow('calibration3',gray)

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

# If found, add object points, image points
if ret == True:
#     print(objp)
#     print(corners)
    objpoints.append(objp)
    imgpoints.append(corners)

    # Draw and display the corners
#     img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
#     cv2.imshow('img',img)
#             cv2.waitKey(500)

#     image = mpimg.imread('camera_cal/calibration1.jpg')
print(img.shape)
print(img.shape[1::-1])
# return cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

img_undistort = undistort_img(img, mtx, dist)

cv2.imshow('img_undistort', img_undistort)


# cwd = os.getcwd()
# # Make a list of calibration im1ages
# images = glob.glob(cwd + './camera_cal/calibration*.jpg')



# # Make a list of calibration images
# images = glob.glob(cwd + '/camera_cal/calibration*.jpg')

# for index, itrImg in enumerate(images):
#     if (index == 1) :
#         print(itrImg)
# #         img = cv2.imread(itrImg)
#         img = cv2.imread(cwd + '/camera_cal/calibration3.jpg')

#         plt.imshow(img)
# #         cv2.imshow('img',img)
#         undist = cv2.undistort(img, mtx, dist, None, mtx)
#         plt.imshow(undist)




D:\hzf\udacity\project\CarND-Advanced-Lane-Lines
(720, 1280)
(720, 1280, 3)
(1280, 720)
